In [1]:
from lxml import etree

In [2]:

l_path = ["/home/ziane212/crisco_work/ressources/1190_prose/1190_Fin_12e_prose_historique_AN_H-T.xml",
          "/home/ziane212/crisco_work/ressources/1487_cronicque/1487_Cronicque_H-T.xml",
          "/home/ziane212/crisco_work/ressources/1862_histoire/1862_Histoire_de_la_Normandie_ancienne_H-T.xml"]                             

path_table = "/home/ziane212/Téléchargements/nsubj_no_det.tsv"

In [3]:
def make_form(w):
    form = ""
    if len(list(w))!=0:
        if list(w)[0].tag == 'choice':
            form = list(w)[0][1].text
        else:
            for child in w.iter():
                if child.text:
                    form += child.text
                if child.tail:
                    form += child.tail
    else:
        form = w.text
    
    return form

def extract_SUBJnoDET(path_in):
    tree = etree.parse(open(path_in, encoding='utf-8'))
    root = tree.getroot()

    list_results = []

    filename = path_in.split('/')[len(path_in.split('/'))-1]

    for book in root.findall('.//div[@type="book"]'):
        book_nb = book.get('n')
        book_ch = int(book_nb)
        for chapter in book.findall('.//div[@type="chapter"]'):
            chapter_nb = chapter.get('n')
            chapter_ch = int(chapter_nb)
            for section in chapter.findall('.//div[@type="section"]'):
                section_nb = section.get('n')
                section_ch = int(section_nb)
                for para in section.findall('.//p'):
                    para_nb = para.get('n')
                    para_ch = int(para_nb)

                    for id_, s in enumerate(para.findall('.//s')):
                        sentence_nb = s.get('n')
                        sentence_ch = int(sentence_nb)

                        sent_id = book_nb+'_'+chapter_nb+'_'+section_nb+'_'+para_nb+'_'+sentence_nb

                        for w in s.findall('.//w'):        
                            if w.get('udpos') == 'NOUN' and w.get('function') == 'nsubj':
                                id_gov = w.get('n')

                                var = False

                                for w2 in para.findall('.//s')[id_]:

                                    id_dep = w2.get('n')

                                    if w2.get('head') == id_gov and w2.get('udpos') == 'DET':
                                        var = True

                                    elif w2.get('head') == id_gov and id_dep < id_gov and w2.get('function') == 'nummod':
                                        var = True

                                    elif w2.get('head') == id_gov and w2.get('udpos') == 'PROPN' and w2.get('function') == 'flat':
                                        var = True

                                    elif w2.get('head') == id_gov and id_dep > id_gov and w2.get('function') == 'appos':
                                        var = True

                                if var == False:
                                    sent = []

                                    l_context = []
                                    r_context = []
                                    for w3 in s.findall('.//w')[:int(id_gov)-1]:
                                        form = make_form(w3)

                                        l_context.append(form)

                                    for w3 in s.findall('.//w')[int(id_gov):]:
                                        form = make_form(w3)

                                        r_context.append(form)

                                    sent.append(filename)
                                    sent.append(sent_id)
                                    cut_l_context = (len(l_context)-1)-9
                                    sent.append(' '.join(l_context[cut_l_context:len(l_context)]))
                                    sent.append(s.findall('.//w')[int(id_gov)-1].text)
                                    sent.append(' '.join(r_context[:9]))
                                    #dev print(sent)
                                    list_results.append(sent)

    return list_results


         

In [18]:
list_sub = ['acl', 'acl:relcl', 'advcl', 'ccomp', 'xcomp']

def extract_noSUBJ(path_in):
    tree = etree.parse(open(path_in, encoding='utf-8'))
    root = tree.getroot()

    list_results = []

    filename = path_in.split('/')[len(path_in.split('/'))-1]

    for book in root.findall('.//div[@type="book"]'):
        book_nb = book.get('n')
        book_ch = int(book_nb)
        for chapter in book.findall('.//div[@type="chapter"]'):
            chapter_nb = chapter.get('n')
            chapter_ch = int(chapter_nb)
            for section in chapter.findall('.//div[@type="section"]'):
                section_nb = section.get('n')
                section_ch = int(section_nb)
                for para in section.findall('.//p'):
                    para_nb = para.get('n')
                    para_ch = int(para_nb)

                    for id_, s in enumerate(para.findall('.//s')):
                        sentence_nb = s.get('n')
                        sentence_ch = int(sentence_nb)

                        sent_id = book_nb+'_'+chapter_nb+'_'+section_nb+'_'+para_nb+'_'+sentence_nb

                        for w in s.findall('.//w'):        
                            if w.get('udpos') == 'VERB':
                                id_gov = w.get('n')
                                head_gov = w.get('head')
                                var = False

                                for w2 in para.findall('.//s')[id_]:

                                    id_dep = w2.get('n')

                                    if w2.get('head') == id_gov and (w2.get('function') == 'nsubj' or w2.get('function') == 'expl'):
                                        var = True
                                        
                                    elif w.get('function') == 'conj':
                                        
                                        for w3 in para.findall('.//s')[id_]:

                                            if head_gov == w3.get('head') and w3.get('function') == 'nsubj':
                                                var = True
                                                
                                if w.get('function') == 'root':
                                    type_s = 'principale'
                                elif w.get('function') in list_sub:
                                    type_s = 'subordonnée_'+w.get('function')
                                        

                                if var == False:
                                    sent = []

                                    l_context = []
                                    r_context = []
                                    for w3 in s.findall('.//w')[:int(id_gov)-1]:
                                        form = make_form(w3)

                                        l_context.append(form)

                                    for w3 in s.findall('.//w')[int(id_gov):]:
                                        form = make_form(w3)

                                        r_context.append(form)

                                    sent.append(filename)
                                    sent.append(sent_id)
                                    cut_l_context = (len(l_context)-1)-9
                                    sent.append(' '.join(l_context[cut_l_context:len(l_context)]))
                                    sent.append(s.findall('.//w')[int(id_gov)-1].text)
                                    sent.append(' '.join(r_context[:9]))
                                    sent.append('Nul')
                                    sent.append(type_s)
                                    #dev print(sent)
                                    list_results.append(sent)
                                    
                                else:
                                    sent = []

                                    l_context = []
                                    r_context = []
                                    for w3 in s.findall('.//w')[:int(id_gov)-1]:
                                        form = make_form(w3)

                                        l_context.append(form)

                                    for w3 in s.findall('.//w')[int(id_gov):]:
                                        form = make_form(w3)

                                        r_context.append(form)

                                    sent.append(filename)
                                    sent.append(sent_id)
                                    cut_l_context = (len(l_context)-1)-9
                                    sent.append(' '.join(l_context[cut_l_context:len(l_context)]))
                                    sent.append(s.findall('.//w')[int(id_gov)-1].text)
                                    sent.append(' '.join(r_context[:9]))
                                    sent.append('Exprimé')
                                    sent.append(type_s)
                                    #dev print(sent)
                                    list_results.append(sent)
                                    

    return list_results

In [19]:
for path_in in l_path:
    
    list_results = extract_noSUBJ(path_in)
    
    with open(path_table, 'a', encoding="utf-8") as tsv:
        for i in list_results:
            print(i)
            
            #tsv.write('\t'.join(i)+'\n')

['1190_Fin_12e_prose_historique_AN_H-T.xml', '1_1_1_1_1', '— Cestes leis furent primes', 'establies', 'e tenues en tens le rey Edgar , ke', 'Exprimé', 'principale']
['1190_Fin_12e_prose_historique_AN_H-T.xml', '1_1_1_1_1', 'primes establies e', 'tenues', 'en tens le rey Edgar , ke fu ael', 'Exprimé', 'principale']
['1190_Fin_12e_prose_historique_AN_H-T.xml', '1_1_1_1_2', 'e Seint Edward sun fiz les', 'tint', 'tant cume fu rei quatre anz e quant fu', 'Nul', 'principale']
['1190_Fin_12e_prose_historique_AN_H-T.xml', '1_1_1_1_2', 'tint tant cume fu rei quatre anz e quant fu', 'ocis', 'par sa marastre , ke li fist murdrir a', 'Nul', 'principale']
['1190_Fin_12e_prose_historique_AN_H-T.xml', '1_1_1_1_2', 'e quant fu ocis par sa marastre , ke li', 'fist', 'murdrir a tort sanz copes e par envie pur', 'Nul', 'principale']
['1190_Fin_12e_prose_historique_AN_H-T.xml', '1_1_1_1_2', 'quant fu ocis par sa marastre , ke li fist', 'murdrir', 'a tort sanz copes e par envie pur ceo', 'Nul', 'subordonné

['1487_Cronicque_H-T.xml', '1_1_1_1_1', 'Aprez que le roy Philippe de France', 'surnommé', 'Auguste eut conquis , réduit , et remis en', 'Nul', 'subordonnée_advcl']
['1487_Cronicque_H-T.xml', '1_1_1_1_1', 'Auguste eut', 'conquis', ', réduit , et remis en sa main et', 'Exprimé', 'subordonnée_advcl']
['1487_Cronicque_H-T.xml', '1_1_1_1_1', 'Aprez que le roy Philippe de France surnommé Auguste eut conquis ,', 'réduit', ', et remis en sa main et soubz la', 'Exprimé', 'subordonnée_advcl']
['1487_Cronicque_H-T.xml', '1_1_1_1_1', 'roy Philippe de France surnommé Auguste eut conquis , réduit , et', 'remis', 'en sa main et soubz la couronne de France', 'Exprimé', 'subordonnée_advcl']
['1487_Cronicque_H-T.xml', '1_1_1_1_1', 'Normendie et plusieurs autres grans terres et seigneuries , et', 'retraictes', "des mains des roys d' Engleterre qui par long", 'Nul', 'subordonnée_advcl']
['1487_Cronicque_H-T.xml', '1_1_1_1_1', "des mains des roys d' Engleterre qui par long temps", 'passé', 'les avoient te

['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_1_1_1', "L' histoire de la Normandie", 'paraît', "ne commencer qu' à l' époque où , devenue", 'Exprimé', 'principale']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_1_1_1', 'Normandie paraît ne', 'commencer', "qu' à l' époque où , devenue par la", 'Nul', 'subordonnée_xcomp']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_1_1_1', "Normandie paraît ne commencer qu' à l' époque où ,", 'devenue', 'par la conquête des hommes du Nord ou Scandinaves', 'Nul', 'subordonnée_advcl']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_1_1_1', 'du Nord ou Scandinaves , une souveraineté particulière , elle', 'offre', 'à la postérité de grands traits et des événements', 'Exprimé', 'subordonnée_advcl']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_1_1_2', 'qui ont', 'précédé', "l' invasion des pirates de la Norwége nous a", 'Exprimé', 'subordonnée_acl:relcl']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1

['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_8_35_2', 'Mais Henri ne put', 'seconder', 'ces braves ;', 'Exprimé', 'principale']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_8_35_3', 'Cavey et Louis', 'périrent', 'victimes de leur dévouement à sa cause .', 'Exprimé', 'principale']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_8_36_1', 'Après avoir vainement', 'tenté', 'de prendre Paris , Henri fut rappelé en Normandie', 'Nul', 'subordonnée_advcl']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_8_36_1', 'Après avoir vainement tenté de', 'prendre', 'Paris , Henri fut rappelé en Normandie , dans', 'Nul', 'subordonnée_xcomp']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_8_36_1', 'Après avoir vainement tenté de prendre Paris , Henri fut', 'rappelé', "en Normandie , dans les plaines d' Ivry ,", 'Exprimé', 'principale']
['1862_Histoire_de_la_Normandie_ancienne_H-T.xml', '1_1_8_36_1', "Normandie , dans les plaines d' Ivry , où il", 'remporta',